In [3]:
DATA_ROOT = 'data/'

In [2]:
# DAY01
nums = [int(x) for x in open(DATA_ROOT + '01.txt', 'r').readlines()]

reg = {}

TOTAL = 2020
for i, n in enumerate(nums):
    rest = TOTAL - n
    if rest in reg:
        print(f'{n} * {rest} = {n * rest}')
    reg[n] = i
    
for i, n1 in enumerate(nums):
    for j in range(i + 1, len(nums)):
        n2 = nums[j]
        rest = TOTAL - n1 - n2
        if rest in reg and reg[rest] > j:
            print(f'{n1} * {n2} * {rest} = {n1 * n2 * rest}')

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [3]:
# DAY02
def parse_line(line):
    parts = line.strip().split()
    counts = list(map(int, parts[0].split('-')))
    return (counts[0], counts[1], parts[1][0], parts[2])

def is_valid1(desc):
    cnt1, cnt2, ch, pwd = desc
    cnt = pwd.count(ch)
    return cnt1 <= cnt <= cnt2

def is_valid2(desc):
    cnt1, cnt2, ch, pwd = desc
    return (pwd[cnt1 - 1] == ch) ^ (pwd[cnt2 - 1] == ch)
    
passwords = list(map(parse_line, open(DATA_ROOT + '02.txt', 'r').readlines()))
res1 = sum([is_valid1(pwd) for pwd in passwords])
res2 = sum([is_valid2(pwd) for pwd in passwords])

print(f'Number of valid passwords: {res1}, {res2}')

Number of valid passwords: 548, 502


In [7]:
# DAY03
import math 

def count_trees(rows, dx, dy):
    h = len(rows)
    if h == 0:
        return 0
    w = len(rows[0])
    x, y = 0, 0
    res = 0
    while y < h:
        res += rows[y][x % w] == '#'
        x += dx
        y += dy
    return res

rows = [x.strip() for x in open(DATA_ROOT + '03.txt', 'r').readlines()]
res1 = count_trees(rows, 3, 1)
print(f'Num trees 1: {res1}')

SLOPES = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
res2 = math.prod((count_trees(rows, x, y) for (x, y) in SLOPES))
print(f'Num trees 2: {res2}')

Num trees 1: 242
Num trees 2: 2265549792


In [85]:
# DAY04
def read_doc(line):
    return dict([x.split(':') for x in line.split()])

REQUIRED_FIELDS = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']

def is_valid1(passport):
    return all(f in passport for f in REQUIRED_FIELDS)

def in_range(int_str, minv, maxv):
    try:
        return minv <= int(int_str) <= maxv
    except:
        return False

def is_valid_height(height_str):
    match = re.match(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$", height_str)
    if match == None:
        return False
    hcm, hin = match.group('cm'), match.group('in')
    if hcm != None:
        return in_range(hcm, 150, 193)
    elif hin != None:
        return in_range(hin, 59, 76)
    else:
        return False

VALID_EYE_COLORS = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

def is_valid2(passport):
    return is_valid1(passport) \
        and in_range(passport['byr'], 1920, 2002) \
        and in_range(passport['iyr'], 2010, 2020) \
        and in_range(passport['eyr'], 2020, 2030) \
        and passport['ecl'] in VALID_EYE_COLORS \
        and re.match(r"^(\d{9})$", passport['pid']) is not None \
        and re.match(r"^#[0-9a-f]{6}$", passport['hcl']) is not None \
        and is_valid_height(passport['hgt'])
    
lines = open(DATA_ROOT + '04.txt', 'r').read()
passports = [read_doc(line) for line in lines.split("\n\n")]

print(f'Valid passports 1: {sum(map(is_valid1, passports))}')
print(f'Valid passports 2: {sum(map(is_valid2, passports))}')

Valid passports 1: 264
Valid passports 2: 224


In [4]:
# DAY05
from functools import reduce

def get_id(seat):
    return reduce(lambda res, c: res * 2 + (c in 'BR'), seat, 0)

lines = [x.strip() for x in open(DATA_ROOT + '05.txt', 'r').readlines()]
ids = [get_id(line) for line in lines]

idset = set(ids)
seats = [id for id in range(1 << len(lines[0]))
    if id not in idset and id + 1 in idset and id - 1 in idset]

print(f'Max ID: {max(ids)}, Seat: {seats[0]}')

Max ID: 801, Seat: 597
